## Інтерактивний Phi 3 Mini 4K Instruct Chatbot з Whisper

### Вступ:
Інтерактивний Phi 3 Mini 4K Instruct Chatbot — це інструмент, який дозволяє користувачам взаємодіяти з демонстрацією Microsoft Phi 3 Mini 4K Instruct за допомогою текстового або аудіовведення. Чатбот можна використовувати для різноманітних завдань, таких як переклад, оновлення погоди та загальне збирання інформації.


In [ ]:
#Install required Python Packages
!pip install accelerate
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install flash-attn --no-build-isolation', env={'FLASH_ATTENTION_SKIP_CUDA_BUILD': "TRUE"}, shell=True
!pip install transformers
!pip install wheel
!pip install gradio
!pip install pydub==0.25.1
!pip install edge-tts
!pip install openai-whisper==20231117
!pip install ffmpeg==1.4
# from IPython.display import clear_output
# clear_output()

In [ ]:
# Checking to see if Cuda support is available 
# Output True = Cuda
# Output False = No Cuda (installing Cuda will be required to run the model on GPU)
import os 
import torch
print(torch.cuda.is_available())


Створіть свій токен доступу Huggingface

Створіть новий токен  
Введіть нову назву  
Виберіть права на запис  
Скопіюйте токен і збережіть його в безпечному місці


Наступний код на Python виконує дві основні задачі: імпортує модуль `os` та встановлює змінну середовища.

1. Імпорт модуля `os`:
   - Модуль `os` у Python забезпечує можливість взаємодії з операційною системою. Він дозволяє виконувати різні завдання, пов’язані з операційною системою, такі як доступ до змінних середовища, робота з файлами та каталогами тощо.
   - У цьому коді модуль `os` імпортується за допомогою оператора `import`. Цей оператор робить функціональність модуля `os` доступною для використання у поточному Python-скрипті.

2. Встановлення змінної середовища:
   - Змінна середовища — це значення, до якого можуть звертатися програми, що працюють в операційній системі. Це спосіб зберігати налаштування конфігурації або іншу інформацію, яка може використовуватися кількома програмами.
   - У цьому коді нова змінна середовища встановлюється за допомогою словника `os.environ`. Ключем словника є `'HF_TOKEN'`, а значення присвоюється зі змінної `HUGGINGFACE_TOKEN`.
   - Змінна `HUGGINGFACE_TOKEN` визначена безпосередньо перед цим фрагментом коду, і їй присвоєно рядкове значення `"hf_**************"` за допомогою синтаксису `#@param`. Цей синтаксис часто використовується у Jupyter-ноутбуках для введення користувацьких даних і налаштування параметрів безпосередньо через інтерфейс ноутбука.
   - Встановивши змінну середовища `'HF_TOKEN'`, до неї можуть звертатися інші частини програми або інші програми, що працюють у тій самій операційній системі.

Загалом, цей код імпортує модуль `os` і встановлює змінну середовища з назвою `'HF_TOKEN'`, використовуючи значення, надане у змінній `HUGGINGFACE_TOKEN`.


In [ ]:
import os
# set the Hugging Face Token from 
# add the Hugging Face Token to the environment variables
HUGGINGFACE_TOKEN = "Enter Hugging Face Key" #@param {type:"string"}
os.environ['HF_TOKEN']HUGGINGFACE_TOKEN

Цей фрагмент коду визначає функцію під назвою clear_output, яка використовується для очищення виводу поточної комірки в Jupyter Notebook або IPython. Давайте розберемо код і зрозуміємо його функціональність:

Функція clear_output приймає один параметр під назвою wait, який є булевим значенням. За замовчуванням wait встановлено як False. Цей параметр визначає, чи повинна функція чекати, поки новий вивід стане доступним для заміни існуючого виводу перед його очищенням.

Сама функція використовується для очищення виводу поточної комірки. У Jupyter Notebook або IPython, коли комірка генерує вивід, наприклад, текст або графічні побудови, цей вивід відображається під коміркою. Функція clear_output дозволяє очистити цей вивід.

Реалізація функції не надана у фрагменті коду, як зазначено через три крапки (...). Три крапки представляють заповнювач для фактичного коду, який виконує очищення виводу. Реалізація функції може включати взаємодію з API Jupyter Notebook або IPython для видалення існуючого виводу з комірки.

Загалом, ця функція забезпечує зручний спосіб очищення виводу поточної комірки в Jupyter Notebook або IPython, що полегшує управління та оновлення відображеного виводу під час інтерактивних сеансів програмування.


In [ ]:
# Download Phi-3-mini-4k-instruct model & Whisper Tiny
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

#whisper for speech to text()
import whisper
select_model ="tiny" # ['tiny', 'base']
whisper_model = whisper.load_model(select_model)

#from IPython.display import clear_output
#clear_output()

Виконання тексту в мовлення (TTS) за допомогою сервісу Edge TTS. Давайте розглянемо відповідні реалізації функцій одну за одною:

1. `calculate_rate_string(input_value)`: Ця функція приймає вхідне значення і обчислює рядок швидкості для голосу TTS. Вхідне значення представляє бажану швидкість мовлення, де значення 1 відповідає нормальній швидкості. Функція обчислює рядок швидкості, віднімаючи 1 від вхідного значення, множачи його на 100, а потім визначаючи знак залежно від того, чи вхідне значення більше або дорівнює 1. Функція повертає рядок швидкості у форматі "{sign}{rate}".

2. `make_chunks(input_text, language)`: Ця функція приймає текст і мову як параметри. Вона розбиває текст на частини відповідно до мовних правил. У цьому випадку, якщо мова — "English", функція розбиває текст на кожній крапці (".") і видаляє будь-які пробіли на початку або в кінці. Потім вона додає крапку до кожної частини і повертає відфільтрований список частин.

3. `tts_file_name(text)`: Ця функція генерує ім'я файлу для аудіофайлу TTS на основі введеного тексту. Вона виконує кілька трансформацій тексту: видаляє крапку в кінці (якщо є), перетворює текст у нижній регістр, видаляє пробіли на початку і в кінці, а також замінює пробіли на підкреслення. Потім вона скорочує текст до максимум 25 символів (якщо довший) або використовує весь текст, якщо він порожній. Нарешті, вона генерує випадковий рядок за допомогою модуля [`uuid`] і об'єднує його зі скороченим текстом, щоб створити ім'я файлу у форматі "/content/edge_tts_voice/{truncated_text}_{random_string}.mp3".

4. `merge_audio_files(audio_paths, output_path)`: Ця функція об'єднує кілька аудіофайлів в один. Вона приймає список шляхів до аудіофайлів і шлях для збереження як параметри. Функція ініціалізує порожній об'єкт `AudioSegment` під назвою [`merged_audio`]. Потім вона ітерує кожен шлях до аудіофайлу, завантажує аудіофайл за допомогою методу `AudioSegment.from_file()` з бібліотеки `pydub` і додає поточний аудіофайл до об'єкта [`merged_audio`]. Нарешті, вона експортує об'єднаний аудіофайл до вказаного шляху у форматі MP3.

5. `edge_free_tts(chunks_list, speed, voice_name, save_path)`: Ця функція виконує операцію TTS за допомогою сервісу Edge TTS. Вона приймає список текстових частин, швидкість мовлення, ім'я голосу та шлях для збереження як параметри. Якщо кількість частин більше 1, функція створює каталог для зберігання окремих аудіофайлів частин. Потім вона ітерує кожну частину, створює команду Edge TTS за допомогою функції `calculate_rate_string()`, імені голосу та тексту частини, і виконує команду за допомогою функції `os.system()`. Якщо виконання команди успішне, вона додає шлях до створеного аудіофайлу до списку. Після обробки всіх частин вона об'єднує окремі аудіофайли за допомогою функції `merge_audio_files()` і зберігає об'єднаний аудіофайл до вказаного шляху. Якщо є лише одна частина, вона безпосередньо створює команду Edge TTS і зберігає аудіо до шляху збереження. Нарешті, вона повертає шлях збереження створеного аудіофайлу.

6. `random_audio_name_generate()`: Ця функція генерує випадкове ім'я аудіофайлу за допомогою модуля [`uuid`]. Вона генерує випадковий UUID, перетворює його на рядок, бере перші 8 символів, додає розширення ".mp3" і повертає випадкове ім'я аудіофайлу.

7. `talk(input_text)`: Ця функція є основною точкою входу для виконання операції TTS. Вона приймає текст як параметр. Спочатку вона перевіряє довжину тексту, щоб визначити, чи є це довгим реченням (600 символів або більше). Залежно від довжини та значення змінної `translate_text_flag`, вона визначає мову і генерує список текстових частин за допомогою функції `make_chunks()`. Потім вона генерує шлях для збереження аудіофайлу за допомогою функції `random_audio_name_generate()`. Нарешті, вона викликає функцію `edge_free_tts()` для виконання операції TTS і повертає шлях збереження створеного аудіофайлу.

Загалом, ці функції працюють разом, щоб розбити введений текст на частини, створити ім'я файлу для аудіофайлу, виконати операцію TTS за допомогою сервісу Edge TTS і об'єднати окремі аудіофайли в один аудіофайл.


In [ ]:
#@title Edge TTS
def calculate_rate_string(input_value):
    rate = (input_value - 1) * 100
    sign = '+' if input_value >= 1 else '-'
    return f"{sign}{abs(int(rate))}"


def make_chunks(input_text, language):
    language="English"
    if language == "English":
      temp_list = input_text.strip().split(".")
      filtered_list = [element.strip() + '.' for element in temp_list[:-1] if element.strip() and element.strip() != "'" and element.strip() != '"']
      if temp_list[-1].strip():
          filtered_list.append(temp_list[-1].strip())
      return filtered_list


import re
import uuid
def tts_file_name(text):
    if text.endswith("."):
        text = text[:-1]
    text = text.lower()
    text = text.strip()
    text = text.replace(" ","_")
    truncated_text = text[:25] if len(text) > 25 else text if len(text) > 0 else "empty"
    random_string = uuid.uuid4().hex[:8].upper()
    file_name = f"/content/edge_tts_voice/{truncated_text}_{random_string}.mp3"
    return file_name


from pydub import AudioSegment
import shutil
import os
def merge_audio_files(audio_paths, output_path):
    # Initialize an empty AudioSegment
    merged_audio = AudioSegment.silent(duration=0)

    # Iterate through each audio file path
    for audio_path in audio_paths:
        # Load the audio file using Pydub
        audio = AudioSegment.from_file(audio_path)

        # Append the current audio file to the merged_audio
        merged_audio += audio

    # Export the merged audio to the specified output path
    merged_audio.export(output_path, format="mp3")

def edge_free_tts(chunks_list,speed,voice_name,save_path):
  # print(chunks_list)
  if len(chunks_list)>1:
    chunk_audio_list=[]
    if os.path.exists("/content/edge_tts_voice"):
      shutil.rmtree("/content/edge_tts_voice")
    os.mkdir("/content/edge_tts_voice")
    k=1
    for i in chunks_list:
      print(i)
      edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{i}" --write-media /content/edge_tts_voice/{k}.mp3'
      print(edge_command)
      var1=os.system(edge_command)
      if var1==0:
        pass
      else:
        print(f"Failed: {i}")
      chunk_audio_list.append(f"/content/edge_tts_voice/{k}.mp3")
      k+=1
    # print(chunk_audio_list)
    merge_audio_files(chunk_audio_list, save_path)
  else:
    edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{chunks_list[0]}" --write-media {save_path}'
    print(edge_command)
    var2=os.system(edge_command)
    if var2==0:
      pass
    else:
      print(f"Failed: {chunks_list[0]}")
  return save_path

# text = "This is Microsoft Phi 3 mini 4k instruct Demo" Simply update the text variable with the text you want to convert to speech
text = 'This is Microsoft Phi 3 mini 4k instruct Demo'  # @param {type: "string"}
Language = "English" # @param ['English']
# Gender of voice simply change from male to female and choose the voice you want to use
Gender = "Female"# @param ['Male', 'Female']
female_voice="en-US-AriaNeural"# @param["en-US-AriaNeural",'zh-CN-XiaoxiaoNeural','zh-CN-XiaoyiNeural']
speed = 1  # @param {type: "number"}
translate_text_flag  = False
if len(text)>=600:
  long_sentence = True
else:
  long_sentence = False

# long_sentence = False # @param {type:"boolean"}
save_path = ''  # @param {type: "string"}
if len(save_path)==0:
  save_path=tts_file_name(text)
if Language == "English" :
  if Gender=="Male":
    voice_name="en-US-ChristopherNeural"
  if Gender=="Female":
    voice_name=female_voice
    # voice_name="en-US-AriaNeural"


if translate_text_flag:
  input_text=text
  # input_text=translate_text(text, Language)
  # print("Translateting")
else:
  input_text=text
if long_sentence==True and translate_text_flag==True:
  chunks_list=make_chunks(input_text,Language)
elif long_sentence==True and translate_text_flag==False:
  chunks_list=make_chunks(input_text,"English")
else:
  chunks_list=[input_text]
# print(chunks_list)
# edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
# from IPython.display import clear_output
# clear_output()
# from IPython.display import Audio
# Audio(edge_save_path, autoplay=True)

from IPython.display import clear_output
from IPython.display import Audio
if not os.path.exists("/content/audio"):
    os.mkdir("/content/audio")
import uuid
def random_audio_name_generate():
  random_uuid = uuid.uuid4()
  audio_extension = ".mp3"
  random_audio_name = str(random_uuid)[:8] + audio_extension
  return random_audio_name
def talk(input_text):
  global translate_text_flag,Language,speed,voice_name
  if len(input_text)>=600:
    long_sentence = True
  else:
    long_sentence = False

  if long_sentence==True and translate_text_flag==True:
    chunks_list=make_chunks(input_text,Language)
  elif long_sentence==True and translate_text_flag==False:
    chunks_list=make_chunks(input_text,"English")
  else:
    chunks_list=[input_text]
  save_path="/content/audio/"+random_audio_name_generate()
  edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
  return edge_save_path


edge_save_path=talk(text)
Audio(edge_save_path, autoplay=True)

Реалізація двох функцій: convert_to_text і run_text_prompt, а також оголошення двох класів: str і Audio.

Функція convert_to_text приймає audio_path як вхідний параметр і транскрибує аудіо в текст за допомогою моделі під назвою whisper_model. Спочатку функція перевіряє, чи встановлений прапорець gpu на True. Якщо так, то whisper_model використовується з певними параметрами, такими як word_timestamps=True, fp16=True, language='English' і task='translate'. Якщо прапорець gpu встановлений на False, whisper_model використовується з параметром fp16=False. Отримана транскрипція зберігається у файл з назвою 'scan.txt' і повертається як текст.

Функція run_text_prompt приймає message і chat_history як вхідні параметри. Вона використовує функцію phi_demo для генерації відповіді від чат-бота на основі вхідного повідомлення. Згенерована відповідь передається у функцію talk, яка конвертує відповідь у аудіофайл і повертає шлях до файлу. Клас Audio використовується для відображення і відтворення аудіофайлу. Аудіо відображається за допомогою функції display з модуля IPython.display, а об'єкт Audio створюється з параметром autoplay=True, щоб аудіо починало відтворюватися автоматично. chat_history оновлюється вхідним повідомленням і згенерованою відповіддю, після чого повертається порожній рядок і оновлений chat_history.

Клас str — це вбудований клас у Python, який представляє послідовність символів. Він надає різні методи для маніпуляції і роботи з рядками, такі як capitalize, casefold, center, count, encode, endswith, expandtabs, find, format, index, isalnum, isalpha, isascii, isdecimal, isdigit, isidentifier, islower, isnumeric, isprintable, isspace, istitle, isupper, join, ljust, lower, lstrip, partition, replace, removeprefix, removesuffix, rfind, rindex, rjust, rpartition, rsplit, rstrip, split, splitlines, startswith, strip, swapcase, title, translate, upper, zfill і багато інших. Ці методи дозволяють виконувати операції, такі як пошук, заміна, форматування і маніпуляція рядками.

Клас Audio — це користувацький клас, який представляє аудіооб'єкт. Він використовується для створення аудіоплеєра в середовищі Jupyter Notebook. Клас приймає різні параметри, такі як data, filename, url, embed, rate, autoplay і normalize. Параметр data може бути numpy-масивом, списком семплів, рядком, що представляє ім'я файлу або URL, або сирими даними PCM. Параметр filename використовується для вказівки локального файлу, з якого завантажуються аудіодані, а параметр url — для вказівки URL, з якого завантажуються аудіодані. Параметр embed визначає, чи повинні аудіодані бути вбудовані за допомогою URI даних або посилатися на оригінальне джерело. Параметр rate визначає частоту семплювання аудіоданих. Параметр autoplay визначає, чи повинно аудіо починати відтворюватися автоматично. Параметр normalize визначає, чи повинні аудіодані бути нормалізовані (перемасштабовані) до максимально можливого діапазону. Клас Audio також надає методи, такі як reload для перезавантаження аудіоданих з файлу або URL, і атрибути, такі як src_attr, autoplay_attr і element_id_attr для отримання відповідних атрибутів аудіоелемента в HTML.

Загалом, ці функції і класи використовуються для транскрибування аудіо в текст, генерації аудіовідповідей від чат-бота, а також для відображення і відтворення аудіо в середовищі Jupyter Notebook.


In [ ]:
#@title Run gradio app
def convert_to_text(audio_path):
  gpu=True
  if gpu:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=True,language='English',task='translate')
  else:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=False,language='English',task='translate')
  with open('scan.txt', 'w') as file:
    file.write(str(result))
  return result["text"]


import gradio as gr
from IPython.display import Audio, display
def run_text_prompt(message, chat_history):
    bot_message = phi_demo(message)
    edge_save_path=talk(bot_message)
    # print(edge_save_path)
    display(Audio(edge_save_path, autoplay=True))

    chat_history.append((message, bot_message))
    return "", chat_history


def run_audio_prompt(audio, chat_history):
    if audio is None:
        return None, chat_history
    print(audio)
    message_transcription = convert_to_text(audio)
    _, chat_history = run_text_prompt(message_transcription, chat_history)
    return None, chat_history


with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Chat with Phi 3 mini 4k instruct")

    msg = gr.Textbox(label="Ask anything")
    msg.submit(run_text_prompt, [msg, chatbot], [msg, chatbot])

    with gr.Row():
        audio = gr.Audio(sources="microphone", type="filepath")

        send_audio_button = gr.Button("Send Audio", interactive=True)
        send_audio_button.click(run_audio_prompt, [audio, chatbot], [audio, chatbot])

demo.launch(share=True,debug=True)


---

**Відмова від відповідальності**:  
Цей документ було перекладено за допомогою сервісу автоматичного перекладу [Co-op Translator](https://github.com/Azure/co-op-translator). Хоча ми прагнемо до точності, будь ласка, майте на увазі, що автоматичні переклади можуть містити помилки або неточності. Оригінальний документ на його рідній мові слід вважати авторитетним джерелом. Для критичної інформації рекомендується професійний людський переклад. Ми не несемо відповідальності за будь-які непорозуміння або неправильні тлумачення, що виникають у результаті використання цього перекладу.
